# Tugas 1 - Crawling Data (detik.com)

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Fungsi untuk mengambil data dari halaman web Detik.com
def get_data(url, kategori):
    try:
        response = requests.get(url)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return

    soup = BeautifulSoup(response.content, "html.parser")
    articles = soup.find_all("article", class_="list-content__item")

    for article in articles:
        try:
            link = article.find("a")["href"]
            article_response = requests.get(link)
            article_response.raise_for_status()
        except (requests.exceptions.RequestException, TypeError) as e:
            print(f"Request for article failed: {e}")
            continue

        article_soup = BeautifulSoup(article_response.content, "html.parser")
        title_element = article_soup.find("h1", class_="detail__title")
        title = title_element.text.strip() if title_element else "Title Not Found"
        date_element = article_soup.find("div", class_="detail__date")
        date = date_element.text.strip() if date_element else "Date Not Found"
        content_element = article_soup.find("div", class_="detail__body-text")
        content = content_element.text.strip() if content_element else "Content Not Found"

        judul.append(title)
        tanggal.append(date)
        isi.append(content)

        print(title)
        time.sleep(1)  # Menambahkan jeda waktu 1 detik antara permintaan artikel

# Membuat list url dan kategori yang akan di-crawl
base_urls = ["https://finance.detik.com/indeks"]
categories = ["Ekonomi"]

# Inisialisasi list untuk menyimpan data
judul = []
tanggal = []
isi = []

# Melakukan iterasi untuk setiap url dan kategori
for base_url, category in zip(base_urls, categories):
    for page in range(1, 4):  # Looping untuk beralih halaman
        url = f"{base_url}/{page}"
        get_data(url, category)
        time.sleep(2)  # Menambahkan jeda waktu 2 detik antara permintaan halaman

# Membuat dataframe dari list data
df = pd.DataFrame({
    "judul": judul,
    "tanggal": tanggal,
    "isi": isi
})

# Menyimpan dataframe ke file CSV
df.to_csv("Crawl-berita.csv", index=False)

# Menampilkan dataframe
print(df)


Makan Bergizi Gratis-Bangun Sekolah Unggulan Masuk Prioritas Prabowo-Gibran
Menaker Akui Jumlah PHK Meningkat, Tembus 46.000!
RI Deflasi 4 Bulan Beruntun, Tanda Pelemahan Daya Beli?
Anggarannya Berkurang Rp117 M, BPIW Ngadu ke DPR
Jokowi Naikkan Tukin Pegawai Komnas HAM, Paling Tinggi Rp 29 Juta!
Cara Daftar QR Code Pertamina untuk Beli Pertalite dan Solar
Cuma Dapat Anggaran Rp 277 M, Erick Thohir: Tidak Sebanding dengan Prestasi
BI Prediksi Ekonomi RI Bisa Tembus 5,6% Tahun Depan
Sri Mulyani Buka-bukaan Anggaran Pilkada 2024
Ini Biang Kerok yang Bikin Warga Jonggol Kesulitan Air Bersih
Antisipasi Kenaikan Bitcoin, Ajaib Kripto Berikan Bonus Transfer 1%
Daftar Tarif Listrik Per kWh yang Berlaku September 2024
Mulai 9 September, Tiket Kereta Bandara Soetta Diskon Rp 16 Ribu!
Jokowi Pamer Kerja Sama Bisnis dengan Afrika Capai Rp 54,3 T di IAF Bali
Erick Thohir Pamer BUMN Beri Setoran ke Negara Rp 1.940 T
Ketahui Syarat dan Cara Pinjam Uang di Shopee dengan SPinjam
Erick Thohir Rapat Bar

In [2]:
from google.colab import files

# Upload file dari komputer
uploaded = files.upload()

Saving Crawl-berita.csv to Crawl-berita (1).csv


In [3]:
import pandas as pd
import io

# Membaca file CSV yang diunggah dan menyimpannya ke dalam DataFrame dengan variabel df
df = pd.read_csv(io.BytesIO(uploaded['Crawl-berita (1).csv']))
df.head(10)

,judul,tanggal,isi
0,Makan Bergizi Gratis-Bangun Sekolah Unggulan M...,"Senin, 02 Sep 2024 13:23 WIB",Jakarta - Kementerian Perencanaan Pembangunan ...
1,"Menaker Akui Jumlah PHK Meningkat, Tembus 46.000!","Senin, 02 Sep 2024 13:07 WIB",Jakarta - Menteri Ketenagakerjaan (Menaker) Id...
2,"RI Deflasi 4 Bulan Beruntun, Tanda Pelemahan D...","Senin, 02 Sep 2024 13:04 WIB",Jakarta - Badan Pusat Statistik (BPS) mengumum...
3,"Anggarannya Berkurang Rp117 M, BPIW Ngadu ke DPR","Senin, 02 Sep 2024 12:58 WIB",Jakarta - Kementerian Pekerjaan Umum dan Perum...
4,"Jokowi Naikkan Tukin Pegawai Komnas HAM, Palin...","Senin, 02 Sep 2024 12:47 WIB",Jakarta - Presiden Joko Widodo (Jokowi) menaik...
5,Cara Daftar QR Code Pertamina untuk Beli Perta...,"Senin, 02 Sep 2024 12:42 WIB",Jakarta - PT Pertamina Patra Niaga mengimbau m...
6,"Cuma Dapat Anggaran Rp 277 M, Erick Thohir: Ti...","Senin, 02 Sep 2024 12:31 WIB",Jakarta - Menteri BUMN Erick Thohir mengusulka...
7,"BI Prediksi Ekonomi RI Bisa Tembus 5,6% Tahun ...","Senin, 02 Sep 2024 12:26 WIB",Jakarta - Bank Indonesia (BI) memprediksi pert...
8,Sri Mulyani Buka-bukaan Anggaran Pilkada 2024,"Senin, 02 Sep 2024 12:16 WIB",Jakarta - Menteri Keuangan Sri Mulyani Indrawa...
9,Ini Biang Kerok yang Bikin Warga Jonggol Kesul...,"Senin, 02 Sep 2024 12:13 WIB",Jakarta - Kekeringan melanda sejumlah wilayah ...
